In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
data_2019 = pd.read_csv("developer_survey_2019/survey_results_public.csv", low_memory=False)

In [3]:
to_drop = ["Respondent", "OpenSource", "CareerSat", "JobSat", "JobSeek", "ResumeUpdate", "SurveyLength", "SurveyEase", "WelcomeChange", "EntTeams", "ScreenName", "LastIn", "SO", "Blockchain"]

In [5]:
def col_drop(df, to_drop):

    df_dropped = df.copy()

    for flag in to_drop:
        df_dropped.drop([x for x in df_dropped.columns if flag in x], axis=1, inplace=True)

    return df_dropped

In [6]:
data_2019 = col_drop(data_2019, to_drop)
print("Success!")

In [11]:
data_2019.head().T

,0,1,2,3,4
MainBranch,I am a student who is learning to code,I am a student who is learning to code,"I am not primarily a developer, but I write co...",I am a developer by profession,I am a developer by profession
Hobbyist,Yes,No,Yes,No,Yes
Employment,"Not employed, and not looking for work","Not employed, but looking for work",Employed full-time,Employed full-time,Employed full-time
Country,United Kingdom,Bosnia and Herzegovina,Thailand,United States,Ukraine
Student,No,"Yes, full-time",No,No,No
EdLevel,Primary/elementary school,"Secondary school (e.g. American high school, G...","Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)"
UndergradMajor,NaN,NaN,Web development or web design,"Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof..."
EduOther,"Taught yourself a new language, framework, or ...",Taken an online course in programming or softw...,"Taught yourself a new language, framework, or ...",Taken an online course in programming or softw...,Taken an online course in programming or softw...
OrgSize,NaN,NaN,100 to 499 employees,100 to 499 employees,"10,000 or more employees"
DevType,NaN,"Developer, desktop or enterprise applications;...","Designer;Developer, back-end;Developer, front-...","Developer, full-stack","Academic researcher;Developer, desktop or ente..."


In [5]:
data_2019.head().T

,0,1,2,3,4
Respondent,1,2,3,4,5
MainBranch,I am a student who is learning to code,I am a student who is learning to code,"I am not primarily a developer, but I write co...",I am a developer by profession,I am a developer by profession
Hobbyist,Yes,No,Yes,No,Yes
OpenSourcer,Never,Less than once per year,Never,Never,Once a month or more often
OpenSource,The quality of OSS and closed source software ...,The quality of OSS and closed source software ...,The quality of OSS and closed source software ...,The quality of OSS and closed source software ...,"OSS is, on average, of HIGHER quality than pro..."
Employment,"Not employed, and not looking for work","Not employed, but looking for work",Employed full-time,Employed full-time,Employed full-time
Country,United Kingdom,Bosnia and Herzegovina,Thailand,United States,Ukraine
Student,No,"Yes, full-time",No,No,No
EdLevel,Primary/elementary school,"Secondary school (e.g. American high school, G...","Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)"
UndergradMajor,NaN,NaN,Web development or web design,"Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof..."


In [ ]:
data_2019 = data_2019[(data_2019["Country"] == "United States") & 
                      (data_2019["Student"] == "No") & 
                      (data_2019["EdLevel"] != "Some college/university study without earning a degree") & 
                      (data_2019["EdLevel"] != "Associate degree") & 
                      (data_2019["Employment"] == "Employed full-time")].copy()

In [ ]:
education_levels_2019 = {"gradHS": "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",
                         "gradUni": "Bachelor’s degree (BA, BS, B.Eng., etc.)",
                         "gradDeg": ["Master’s degree (MA, MS, M.Eng., MBA, etc.)",
                         "Other doctoral degree (Ph.D, Ed.D., etc.)",
                         "Professional degree (JD, MD, etc.)"]}

In [ ]:
for k, v in education_levels_2019.items():
    data_2019[k] = data_2019["EdLevel"].map(lambda x: str(x) in v) * 1

In [ ]:
data_2019.rename(mapper={"ConvertedComp": "ln_salary"}, axis=1, inplace=True)
data_2019 = data_2019[data_2019["ln_salary"] > 0].copy()
data_2019["ln_salary"] = np.log(data_2019["ln_salary"])
data_2019.dropna(subset=["ln_salary"], inplace=True)

In [ ]:
data_2019 = data_2019[["ln_salary"] + [key for key in education_levels_2019]].copy()